In [ ]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key
from config import Token_NOAA
from config import API_Token
import calendar

In [2]:
# https://github.com/crvaden/NOAA_API_v2
# https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
# https://cran.r-project.org/web/packages/rnoaa/rnoaa.pdf
# file:///C:/Users/15124/Downloads/GHCND_documentation.pdf
# https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

# California Avg Temp and Precipitation API

In [ ]:
# from noaa_api_v2 import NOAAData
# api_token = Token_NOAA

# data = NOAAData(api_token)

# categories = data.data_categories(locationid='FIPS:37', sortfield='name')

# for i in categories:
#     print(i)

In [ ]:
#Create average temperature

# Create empty lists to append the data into
dates_temp = []
temps = []
dates_prcp = []
prcp = []

#Identify what years we want Temp and Prcp data for
for year in range(2013, 2017):
    year = str(year)
    #Print out 'working on year' to idenfity if script is running correctly
    print('working on year '+year)
    
    #make the api call for temp and precipitation
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #Load JSON data
    d = json.loads(r.text)
    e = json.loads(p.text)
    #Get TAVG and PRCP data
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    dates_prcp += [item['date'] for item in avg_prcp]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]
    prcp += [item['value'] for item in avg_prcp]
    
# initialize dataframe
df_temp = pd.DataFrame()
df_prcp = pd.DataFrame()

# populate date and average temperature fields (cast string date to datetime)
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
# convert to degrees F
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]
#convert to mm to inches
df_prcp['avgPrcp'] = [float(v)*0.0393701 for v in prcp]
# Average the temp/prcp per day to temp/prcp per month
df_temp = df_temp.set_index('date').resample('M').mean()
df_prcp = df_prcp.set_index('date').resample('M').mean()
# Combine the series into a df
df_temp['avgPrcp']=df_prcp['avgPrcp']


# Washington Avg Temp and Precipitation API

In [19]:
#Create average temperature
dates_temp_W = []
temps_W = []
dates_prcp_W = []
prcp_W = []
names=['Napa', 'Walla', 'Salem']
zip_codes=['USW00023129', 'USW00023129', 'USW00023129']

df_temp_W = pd.DataFrame()
df_prcp_W = pd.DataFrame()


temp_names=['avgTemp', 'avgTemp','avgTemp']
temp=['Napa_temp','Napa_temp','Napa_temp',]
prcp_names=['avgPrcp', 'avgPrcp', 'avgPrcp']
prcp=['Napa_prcp', 'Napa_prcp', 'Napa_prcp']
bigdf = pd.DataFrame()


#for each year from 2015-2019 ...
for i in range(len(zip_codes)):
    zip_=zip_codes[i]
    for year, temps, t, prcps, p in zip(range(1992, 2014),temp_names, temp, prcp_names, prcp):
        for name in names:
            year = str(year)
            print('working on year '+year)

            #make the api call for temp and precipitation
            r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
            p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:{zip_}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
            #Load JSON data
            d = json.loads(r.text)
            e = json.loads(p.text)
    #         print(d)
            #Get TAVG and PRCP data
            avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
            avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
            #get the date field from all average temperature readings
            dates_temp_W += [item['date'] for item in avg_temps]
            dates_prcp_W += [item['date'] for item in avg_prcp]
            #get the actual average temperature from all average temperature readings
            temps_W += [item['value'] for item in avg_temps]
            prcp_W  += [item['value'] for item in avg_prcp]

            #initialize dataframe
            df_temp_W = pd.DataFrame()
            df_prcp_W = pd.DataFrame()

            #populate date and average temperature fields (cast string date to datetime
            df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
            df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
            #convert to degrees F
            df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
            #convert to mm to inches
            df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
            # Average the temp/prcp per day to temp/prcp per month
            df_temp_W = df_temp_W.set_index('date').resample('M').mean()
            df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
            # Combine the series into a df
            df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']          
#             df_temp_W.rename(columns={temps:t}, inplace=True)
#             df_temp_W.rename(columns={prcps:p}, inplace=True)
#             df_temp_W.reset_index(inplace=True)
#             d=df_temp_W[['date', t, p]]
#             bigdf['Date']=d['date']
#             bigdf[t] = d[t]
#             bigdf[p] = d[p]
df_temp_W           

working on year 1992


KeyError: 'results'

In [17]:
df_temp_W

,date,Napa_temp,<Response [200]>
0,2013-04-30,62.678000,0.007874
1,2013-05-31,67.123226,0.213361
2,2013-06-30,68.300000,0.000000
3,2013-07-31,70.316774,0.022860
4,2013-08-31,70.432903,0.000000
5,2013-09-30,72.326000,0.000000
6,2013-10-31,65.950323,0.039370
7,2013-11-30,62.612000,0.261155
8,2013-12-31,57.072258,0.109220


In [ ]:
temp_names=['avgTemp']
temp=['Napa_temp', 'walla']
prcp_names=['avgPrcp']
prcp=['Napa_prcp']
bigdf = pd.DataFrame()


for temps, t, prcps,p in zip(temp_names, temp, prcp_names, prcp):
    df_temp_W.rename(columns={temps:t}, inplace=True)
    df_temp_W.rename(columns={prcps:p}, inplace=True)
    df_temp_W.reset_index(inplace=True)
#     d=df_temp_W[['date', t, p]]
    bigdf['Date']=df_temp_W['date']
    bigdf[t] = df_temp_W[t]
    bigdf[p] = df_temp_W[p]
bigdf
#             d['Date']=pd.to_datetime(d['Date'], errors='coerce')
#             bigdf['Date']=d['Date']
#             bigdf[name] = d[name]

In [22]:
dates_temp_W = []
temps_W = []
dates_prcp_W = []
prcp_W = []
# names=['Napa', 'Walla', 'Salem']
zip_codes=['USW00023129']

df_temp_W = pd.DataFrame()
df_prcp_W = pd.DataFrame()

for year in range(1992, 2017):
    year = str(year)
    print('working on year '+year)

    #make the api call for temp and precipitation
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
    p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
    #Load JSON data
    d = json.loads(r.text)
    e = json.loads(p.text)
    #Get TAVG and PRCP data
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp_W += [item['date'] for item in avg_temps]
    dates_prcp_W += [item['date'] for item in avg_prcp]
    #get the actual average temperature from all average temperature readings
    temps_W += [item['value'] for item in avg_temps]
    prcp_W  += [item['value'] for item in avg_prcp]

#initialize dataframe
df_temp_W = pd.DataFrame()
df_prcp_W = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
#convert to degrees F
df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
#convert to mm to inches
df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
# Average the temp/prcp per day to temp/prcp per month
df_temp_W = df_temp_W.set_index('date').resample('M').mean()
df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()
# Combine the series into a df
df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']
df_temp_W

working on year 1992


KeyError: 'results'

#  Oregon Avg Temp and Precipitation API

In [ ]:
#Create average temperature
dates_temp_O = []
temps_O = []
dates_prcp_O = []
prcp_O = []

#for each year from 2015-2019 ...
for year in range(2013, 2014):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&locationid=ZIP:28801&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
    p = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=ZIP:28801&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token_NOAA})
    #load the api response as a json
    d = json.loads(r.text)
    print(d)
    e = json.loads(p.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    avg_prcp = [item for item in e['results'] if item['datatype']=='PRCP']
    #get the date field from all average temperature readings
    dates_temp_O += [item['date'] for item in avg_temps]
    dates_prcp_O += [item['date'] for item in avg_prcp]
    #get the actual average temperature from all average temperature readings
    temps_O += [item['value'] for item in avg_temps]
    prcp_O += [item['value'] for item in avg_prcp]

#initialize dataframe
df_temp_O = pd.DataFrame()
df_prcp_O = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_O]
df_prcp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_O]
#convert to degrees F
df_temp_O['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_O]
#convert to mm to inches
df_prcp_O['avgPrcp'] = [float(v)*0.0393701 for v in prcp_O]
# Average the temp/prcp per day to temp/prcp per month
df_temp_O = df_temp_O.set_index('date').resample('M').mean()
df_prcp_O = df_prcp_O.set_index('date').resample('M').mean()
# Combine the series into a df
df_temp_O['avgPrcp']=df_prcp_O['avgPrcp']

# Add Lat/Long/Elevation

In [ ]:
# create new columns for Lat/Long/Elevation for the red and white white data
#There are three clusters of wineries. 
#The wineries in California, Washington, and Oregon are within a couple dozen miles of one another in each state.
#So I just used three stations: One for California, one for Washington, one for Oregon


#-------------------------------------------------------Station IDs---------------------------------------------------#
#California_GHCND:
Nappa_Valley_g='GHCND:USC00046074'
# Washington_GHCND:
Walla_Walla_Valley_g='GHCND:GHCND:USC00457015'
# Oregon_GHCND:
Willamette_Valley_g='GHCND:USW00024232'
#--------------------------------------------------------Latitude-----------------------------------------------------#

#Latitude
#California_LAT:
Cali_lat='38.2777'
# Oregon_lat:
Oregon_lat='44.905'
# Washington_Lat:
Washington_lat='46.3119'


#Create a new column in red_USA_df that has latitude for california, washingtion, oregon
latitude=[]
for lat in red_USA_df['regions']:
#California
    if lat =='California':
        latitude.append(Cali_lat)
#Oregon
    elif lat == 'Oregon':
        latitude.append(Oregon_lat)
#Washington
    else:
        latitude.append(Washington_lat)
red_USA_df['lat']=latitude

#Create a new column in white_USA_df that has latitude for california, washingtion, oregon
latitude=[]
for lat in white_USA_df['regions']:
#California
    if lat =='California':
        latitude.append(Cali_lat)
#Oregon
    elif lat == 'Oregon':
        latitude.append(Oregon_lat)
#Washington
    else:
        latitude.append(Washington_lat)
white_USA_df['lat']=latitude

#------------------------------------------------------Longitude-------------------------------------------------------------#

#Longitude
#California_Long:
Cali_long='-122.2647'
# Oregon_Long:
Oregon_long='-123.0011'
# Washington_Long:
Washington_long= '-119.2633'

#Create a new column in red_USA_df that has longitude for california, washingtion, oregon
longitude=[]
for long in red_USA_df['regions']:
#California
    if long =='California':
        longitude.append(Cali_long)
#Oregon
    elif long == 'Oregon':
        longitude.append(Oregon_long)
#Washington
    else:
        longitude.append(Washington_long)
red_USA_df['long']=longitude 

#Create a new column in white_USA_df that has longitude for california, washingtion, oregon
longitude=[]
for long in white_USA_df['regions']:
#California
    if long =='California':
        longitude.append(Cali_long)
#Oregon
    elif long == 'Oregon':
        longitude.append(Oregon_long)
#Washington
    else:
        longitude.append(Washington_long)
white_USA_df['long']=longitude 

#-----------------------------------------------------------------Elevation------------------------------------------------#

#Elevation (feet)
#California_Elevation:
Cali_ele='35.1'
# Oregon_Elevation:
Oregon_ele='205.1'
# Washington_Elevation:
Washington_ele= '631.9'


#Create a new column in red_USA_df that has elevation for california, washingtion, oregon
elevation=[]
for ele in red_USA_df['regions']:
#California
    if ele =='California':
        elevation.append(Cali_ele)
#Oregon
    elif ele == 'Oregon':
        elevation.append(Oregon_ele)
#Washington
    else:
        elevation.append(Washington_ele)
red_USA_df['elevation']=elevation

#Create a new column in white_USA_df that has elevation for california, washingtion, oregon
elevation=[]
for ele in white_USA_df['regions']:
#California
    if ele =='California':
        elevation.append(Cali_ele)
#Oregon
    elif ele == 'Oregon':
        elevation.append(Oregon_ele)
#Washington
    else:
        elevation.append(Washington_ele)
white_USA_df['elevation']=elevation


In [ ]:
df_temp.head(2)

In [ ]:
white_USA_df.head(2)

In [ ]:
for station, year in zip(white_USA_df['appellation'], )

# Add Temp/Precipitation

In [ ]:
# Split the date into columns for days, months, years and then drop days columns for each of the three states
#-------------------------------------------------------------------------------California----------------------------
df_temp.reset_index(inplace= True )
df_temp['day'] = df_temp['date'].dt.day
df_temp['month'] = df_temp['date'].dt.month
df_temp['year'] = df_temp['date'].dt.year

list_months=[]
for x in df_temp['month']:
    list_months.append(calendar.month_name[x])
df_temp['months']=list_months
df_temp.drop(['date', 'day','month'], axis=1, inplace=True)
df_temp
#----------------------------------------------------------------------------------Oregon-------------------------------
# df_temp_O.reset_index(inplace= True )
# df_temp_O['day'] = df_temp_O['date'].dt.day
# df_temp_O['month'] = df_temp_O['date'].dt.month
# df_temp_O['year'] = df_temp_O['date'].dt.year

# list_months=[]
# for x in df_temp_O['month']:
#     list_months.append(calendar.month_name[x])
# df_temp_O['months']=list_months
# df_temp_O.drop(['date', 'day','month'], axis=1, inplace=True)
# #-----------------------------------------------------------------------------------Washington---------------------
# df_temp_W.reset_index(inplace= True )
# df_temp_W['day'] = df_temp_W['date'].dt.day
# df_temp_W['month'] = df_temp_W['date'].dt.month
# df_temp_W['year'] = df_temp_W['date'].dt.year

# list_months=[]
# for x in df_temp_W['month']:
#     list_months.append(calendar.month_name[x])
# df_temp_W['months']=list_months
# df_temp_W.drop(['date', 'day','month'], axis=1, inplace=True)

In [ ]:
df_temp.head(2)

In [ ]:
red_USA_df

In [ ]:
calender_years=[2013, 2014,2015, 2016, 2017]
calender_months=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

#### California

In [ ]:
# temp_2013=[]
# temp_2014=[]
# temp_2015=[]
# temp_2016=[]
# temp_2017=[]
# temp_2018=[]
# garbage=[]
# prcp_2013=[]
# prcp_2014=[]
# prcp_2015=[]
# prcp_2016=[]
# prcp_2017=[]
# prcp_2018=[]

# for temp, prcp, year, month in zip(df_temp['avgTemp'], df_temp['avgPrcp'], df_temp['year'], df_temp['months']):
#     if year==2013:
#         temp_2013.append(temp)    
#     elif year==2014:
#         temp_2014.append(temp)
#     elif year==2015:
#         temp_2015.append(temp)
#     elif year==2016:
#         temp_2016.append(temp)
#     elif year==2017:
#         temp_2017.append(temp)
#     else:
#         garbage.append(temp)

# for temp, prcp, year, month in zip(df_temp['avgTemp'], df_temp['avgPrcp'], df_temp['year'], df_temp['months']):
#     if year==2013:
#         prcp_2013.append(temp)    
#     elif year==2014:
#         prcp_2014.append(temp)
#     elif year==2015:
#         prcp_2015.append(temp)
#     elif year==2016:
#         prcp_2016.append(temp)
#     elif year==2017:
#         prcp_2017.append(temp)
#     else:
#         garbage.append(temp)
# temp_2013

In [ ]:
# march_2016=[]
# april_2016=[]
# may_2016=[]
# june_2016=[]
# july_2016=[]
# august_2016=[]
# september_2016=[]


# for region,vint,year  in zip(white_USA_df['regions'],white_USA_df['vintage'],temp_2016):
#     if region=='California' and vint==2016:
#         march_2016.append(temp_2016[3])
#     elif region=='California' and vint==2016:
#         april_2016.append(temp_2016[4])
#     elif region=='California' and vint==2016:
#         may_2016.append(temp_2016[5])        
#     elif region=='California' and vint==2016:
#         june_2016.append(temp_2016[6])        
#     elif region=='California' and vint==2016:
#         july_2016.append(temp_2016[7])        
#     elif region=='California' and vint==2016:
#         august_2016.append(temp_2016[8])        
#     elif region=='California' and vint==2016:
#         september_2016.append(temp_2016[9])   
#     else:
#         garbage.append(0)
# march_2016        

In [ ]:
# list_temp_march=[]
# all_months=[]

# for region,vint,year, temp, month, cyears, cmonths  in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months'], calender_years, calender_months):
#     if region=='California' and vint==2016:
#             list_temp_march.append(temp)
#         else:
#             list_temp_march.append(0)
#     else:list_2016=[]
# march_2016=[]
# april_2016=[]
# may_2016=[]
# june_2016=[]
# july_2016=[]
# august_2016=[]
# september_2016=[]
# for region,vint,years,temps,month in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if vint==2016 and region=='Calfornia' and month=='March':
#         march_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='April':
#         april_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='May':
#         may_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='June':
#         june_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='July':
#         july_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='August':
#         august_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='September':
#         september_2016.append(temps) 
#         list_temp_march.append(1)
# list_temp_march
        

In [ ]:
# list_2016=[]
# march_2016=[]
# april_2016=[]
# may_2016=[]
# june_2016=[]
# july_2016=[]
# august_2016=[]
# september_2016=[]
# for region,vint,years,temps,month in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if vint==2016 and region=='Calfornia' and month=='March':
#         march_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='April':
#         april_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='May':
#         may_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='June':
#         june_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='July':
#         july_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='August':
#         august_2016.append(temps)
#     elif vint==2016 and region=='Calfornia' and month=='September':
#         september_2016.append(temps) 

In [ ]:
# white_USA_df.head(2)

In [ ]:
# # Trying to add columns for weather data for each month

# list_temp_march=[]
# # def monthData (year, month, region, vintage):
# for u,v,x,y,z in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if v==2016 and v==2016 and u=='California'and z=='March':
#         list_temp_march.append(y)
#     #elif y==z and y==2016 and x=='Oregon'and v=='April':
#         #list_temp_march.append(u)
#     else:
#         list_temp_march.append(0)
            
# # white_USA_df['march']= list_temp_march         
# # white_USA_df['march']=white_USA_df['march'].apply(monthData)
# # list_temp_march    

In [ ]:
# # Trying to add columns for weather data for each month
# list_temp_march=[]
# march_2016=[]
# april_2016=[]





# # def monthData (year, month, region, vintage):
# for u,v,x,y,z in zip(white_USA_df['regions'],white_USA_df['vintage'],df_temp['year'], df_temp['avgTemp'], df_temp['months']):
#     if v==2016 and v==2016 and u=='California':
#         list_temp_march.append(y)
#     #elif y==z and y==2016 and x=='Oregon'and v=='April':
#         #list_temp_march.append(u)
#     else:
#         list_temp_march.append(0)
            
# # white_USA_df['march']= list_temp_march         
# # white_USA_df['march']=white_USA_df['march'].apply(monthData)
# list_temp_march

In [ ]:
sorted_red_USA_df=red_USA_df.sort_values(by=['regions'], ascending=False)
sorted_red_USA_df=sorted_red_USA_df[sorted_red_USA_df['regions']=='Oregon']
sorted_red_USA_df=sorted_red_USA_df.sort_values(by=['vintage'], ascending=True)
sorted_red_USA_df.head(50)

Imported dependencies<p>
Used the Global Wine Score API to import data for red and white wine in the USA<p>
Imported weather data for California, Oregon, Washington<p>
Added new columns Latitude, Longitude, and Elevation to red and white wine dataframes<p>
Finally I am trying to add new columns for monthly weather data to the red and white wine dataframes<p>

In [ ]:
# # .apply() method
# white_USA_df['march']=[]

#axis=1 for columns
#default--->result_type=None means that final return type is inferred (default)
#raw is false means that it will be a series, True is it is an array